In [1]:
import random
from inspect import signature
from operator import add, mul, itemgetter, attrgetter, methodcaller
from functools import reduce, partial
from collections import namedtuple

### Create and test a function, then read its __doc__ and check its type

In [2]:
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [3]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [4]:
factorial.__doc__

'returns n!'

In [5]:
type(factorial)

function

### Use function through a different name, and pass function as argument

In [6]:
fact = factorial
fact

<function __main__.factorial(n)>

In [7]:
fact(5)

120

In [8]:
map(factorial, range(10))

In [9]:
list(map(factorial, range(10)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

### Sorting a list of words by length

In [10]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

### Sorting a list of words by their reversed spelling

In [11]:
def reverse(word):
    return word[::-1]

In [12]:
reverse('testing')

'gnitset'

In [13]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### Lists of factorials produced with map and filter compared to alternatives coded as list comprehensions

In [14]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [15]:
# Same operation, with a list comprehension.
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [16]:
list(map(factorial, filter(lambda n: n % 2, range(6))))

[1, 6, 120]

In [17]:
# Same operation, with a list comprehension.
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

### Sum of integers up to 99 performed with reduce and sum

In [18]:
%%timeit
reduce(add, range(10000))

806 µs ± 92.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [19]:
%%timeit
sum(range(10000))

275 µs ± 4.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Sorting a list of words by their reversed spelling using lambda

In [20]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## Callable Objects

In [21]:
abs, str, 13

(<function abs(x, /)>, str, 13)

In [22]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

### A BingoCage does one thing: picks items from a shuffled list

In [23]:
class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        return self.pick()

In [24]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [25]:
callable(bingo)  # bingo is a callable

True

In [26]:
bingo()

1

## Function Introspection

In [27]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [28]:
def upper_case_name(obj):
    return ("%s %s" % (obj.first_name, obj.last_name)).upper()

In [29]:
upper_case_name.short_description = 'Customer name'

In [30]:
set(dir(upper_case_name)) - set(dir(factorial))

{'short_description'}

### Listing attributes of functions that don’t exist in plain instances

In [31]:
class C: pass

In [32]:
obj = C()

In [33]:
def func(): pass

In [34]:
sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## From Positional to Keyword-Only Parameters
### tag generates HTML; a keyword-only argument cls is used to pass “class” attributes as a workaround because class is a keyword in Python

In [35]:
def tag(name, *content, cls=None, **attrs):
    """Generate one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                            for attr, value
                            in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                        (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [36]:
tag('br')

'<br />'

In [37]:
tag('p', 'hello')

'<p>hello</p>'

In [38]:
print(tag('p', 'hello', 'world'))

<p>hello</p>
<p>world</p>


In [39]:
tag('p', 'hello', id=33)

'<p id="33">hello</p>'

In [40]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [41]:
tag(content='testing', name="img")

'<img content="testing" />'

In [42]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
            'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [43]:
def f(a, *, b):
    return a, b

In [44]:
f(1, b=2)  # b, here can only be passed as keyword argument

(1, 2)

## Retrieving Information About Parameters

### Function to shorten a string by clipping at a space near the desired length

In [45]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
    if space_before >= 0:
        end = space_before
    else:
        space_after = text.rfind(' ', max_len)
    if space_after >= 0:
        end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

### Extracting information about the function arguments

In [46]:
clip.__defaults__

(80,)

In [47]:
clip.__code__

<code object clip at 0x000002568F0BE920, file "C:\Users\Dell\AppData\Local\Temp/ipykernel_7860/1732165803.py", line 1>

In [48]:
# The argument names appear in __code__.co_varnames, but that 
# also includes the names of the local variables created in the 
# body of the function.
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

### Extracting the function signature

In [49]:
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [50]:
str(sig)

'(text, max_len=80)'

In [51]:
sig.parameters

mappingproxy({'text': <Parameter "text">, 'max_len': <Parameter "max_len=80">})

In [52]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


### Binding the function signature from the tag function in Example 5-10 to a dict of arguments

In [53]:
sig = signature(tag)
my_tag = {
    'name': 'img',
    'title': 'Sunset Boulevard',
    'src': 'sunset.jpg',
    'cls': 'framed'
}

In [54]:
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

In [55]:
for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [56]:
del my_tag['name']

In [57]:
try:
    bound_args = sig.bind(**my_tag)
except TypeError:
    print('missing a required argument: name')

missing a required argument: name


## Function Annotations

### Annotated clip function

In [58]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
    if space_before >= 0:
        end = space_before
    else:
        space_after = text.rfind(' ', max_len)
    if space_after >= 0:
        end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

In [59]:
clip.__annotations__

{'text': str, 'max_len': 'int > 0', 'return': str}

### Extracting annotations from the function signature

In [60]:
sig = signature(clip)
sig.return_annotation

str

In [61]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
'int > 0'     : max_len = 80


## The operator Module

### Factorial implemented with reduce and an anonymous function

In [62]:
def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

### Factorial implemented with reduce and operator.mul

In [63]:
def fact(n):
    return reduce(mul, range(1, n+1))

In [64]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

In [65]:
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [66]:
cc_name = itemgetter(1, 0)

In [67]:
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


### Demo of attrgetter to process a previously defined list of namedtuple called metro_data

In [68]:
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

In [69]:
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) #
                for name, cc, pop, (lat, long) in metro_data]

In [70]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [71]:
metro_areas[0].coord.lat

35.689722

In [72]:
name_lat = attrgetter('name', 'coord.lat')
name_lat

operator.attrgetter('name', 'coord.lat')

In [73]:
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


### Demo of methodcaller: second test shows the binding of extra arguments

In [74]:
s = 'The time has come'
uppercase = methodcaller('upper')
uppercase(s)  # Same as str.upper(s)

'THE TIME HAS COME'

In [75]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)  ## # Same as str.repolace(' ', '-')

'The-time-has-come'

## Freezing Arguments with functools.partial

In [76]:
triple = partial(mul, 3)
triple(7)

21

In [77]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

### Demo of partial applied to the function tag

In [78]:
picture = partial(tag, 'img', cls='pic-frame')
picture(src='wumpus.jpeg')
picture

functools.partial(<function tag at 0x000002568F0E4310>, 'img', cls='pic-frame')

In [79]:
picture.func

<function __main__.tag(name, *content, cls=None, **attrs)>

In [80]:
picture.args

('img',)

In [81]:
picture.keywords

{'cls': 'pic-frame'}